## Install and import packages

In [1]:
%pip install pandas numpy psycopg2 ipywidgets

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import time
import ipywidgets as widgets
import psycopg2
from IPython.display import display
import re

## Connect to DB

In [7]:
DB_NAME = "neondb"
DB_USER = "andrea"
DB_PASS = "Sapienza2025!"
DB_HOST = "ep-old-wind-a505sf8m-pooler.us-east-2.aws.neon.tech"
DB_PORT = "5432"


def get_db_connection():
    return psycopg2.connect(
        dbname=DB_NAME, user=DB_USER, password=DB_PASS,
        host=DB_HOST, port=DB_PORT
    )


conn = psycopg2.connect(
    dbname=DB_NAME, user=DB_USER, password=DB_PASS,
    host=DB_HOST, port=DB_PORT
)

## Insert your personal information

In [8]:
import re
import ipywidgets as widgets

# Connessione al database
conn = get_db_connection()

cur = conn.cursor()
cur.execute('''SELECT DISTINCT "FORMATTED_EXPERIENCE_LEVEL" FROM public."POSTING" WHERE "FORMATTED_EXPERIENCE_LEVEL" IS NOT NULL ORDER BY "FORMATTED_EXPERIENCE_LEVEL" ASC''')
experience_levels = [row[0] for row in cur.fetchall()]
cur.close()

cur = conn.cursor()
cur.execute('''SELECT DISTINCT "ID", "SKILL_NAME" FROM public."SKILL" WHERE "SKILL_NAME" IS NOT NULL ORDER BY "SKILL_NAME" ASC''')
skill_data = cur.fetchall()
skill_dict = {row[1]: row[0] for row in skill_data}  # Dizionario {skill_name: skill_id}
skill_names = list(skill_dict.keys())
cur.close()

# Pattern per validare l'email
email_pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'

# Creazione widget
name_input = widgets.Text(placeholder="Insert your name")
surname_input = widgets.Text(placeholder="Insert your surname")
email_input = widgets.Text(placeholder="Insert your email")
experience_input = widgets.Dropdown(options=experience_levels, description="Experience:")
skill_input = widgets.SelectMultiple(options=skill_names, description="Skills:")
button = widgets.Button(description="Insert Data", button_style="success")

output = widgets.Output()

def insert_data(b):
    with output:
        output.clear_output()
        name = name_input.value
        surname = surname_input.value
        email = email_input.value
        experience = experience_input.value
        selected_skills = skill_input.value  # Lista delle skill selezionate
        
        # Validazione email
        if not re.match(email_pattern, email):
            print("Warning: Invalid email format. Please enter a valid email address.")
            email_input.value = ""
            return
        
        try:
            cur = conn.cursor()
            # Inserimento utente
            query_user = '''INSERT INTO public."USERS" ("NAME", "SURNAME", "MAIL", "EXPERIENCE") 
                            VALUES (%s, %s, %s, %s) RETURNING "USER_ID"'''
            cur.execute(query_user, (name, surname, email, experience))
            user_id = cur.fetchone()[0]  # Recupero USER_ID
            conn.commit()
            
            # Inserimento skill per l'utente
            if selected_skills:
                query_skill = '''INSERT INTO public."USER_SKILL" ("USER_ID", "SKILL_ID") VALUES (%s, %s)'''
                for skill_name in selected_skills:
                    skill_id = skill_dict[skill_name]
                    cur.execute(query_skill, (user_id, skill_id))
                
                conn.commit()
            
            cur.close()
            print("Data inserted successfully!")
        except Exception as e:
            print(f"Error while connecting to database: {e}")

# Collegamento del bottone all'evento
button.on_click(insert_data)

# Visualizzazione dei widget
display(widgets.VBox([widgets.Label("Name:"), name_input,
                      widgets.Label("Surname:"), surname_input,
                      widgets.Label("Email:"), email_input,
                      experience_input,
                      skill_input,
                      button, output]))

UndefinedColumn: column "ID" does not exist
LINE 1: SELECT DISTINCT "ID", "SKILL_NAME" FROM public."SKILL" WHERE...
                        ^


## Some interesting insight/question that can be extracted from data:


Most In-Demand Job Titles

```SQL 
SELECT "job_title", COUNT(*) AS num_postings
FROM public."postings"
GROUP BY "job_title"
ORDER BY num_postings DESC
LIMIT 10;
```

Most Hiring Companies

```SQL 
SELECT "company_name", COUNT(*) AS num_postings
FROM public."postings"
GROUP BY "company_name"
ORDER BY num_postings DESC
LIMIT 10;
```

Job Locations with Highest Demand

```SQL 
SELECT "job_location", COUNT(*) AS num_postings
FROM public."postings"
GROUP BY "job_location"
ORDER BY num_postings DESC
LIMIT 10;
```

Average Salary for Each Job Title (if salary info is available)

```SQL 
SELECT "job_title", AVG("salary") AS avg_salary
FROM public."postings"
WHERE "salary" IS NOT NULL
GROUP BY "job_title"
ORDER BY avg_salary DESC
LIMIT 10;
```

Remote vs On-Site Jobs

```SQL
SELECT "work_type", COUNT(*) AS num_postings
FROM public."postings"
WHERE "work_type" IN ('Remote', 'On-site')
GROUP BY "work_type";
```

Most Common Skills Required

```SQL
SELECT "skills", COUNT(*) AS num_occurrences
FROM public."postings"
WHERE "skills" IS NOT NULL
GROUP BY "skills"
ORDER BY num_occurrences DESC
LIMIT 10;
```

## 1. Select all the ...

## 2. Select all the posting job for company name given by input and with requested location and for a specific job location

In [21]:
company_input = widgets.Text(placeholder="Insert the company name")
location_input = widgets.Text(placeholder="Insert the preferred location")
job_title_input = widgets.Text(placeholder="Insert the job title")
search_button = widgets.Button(description="Look for job posting", button_style="primary")
output = widgets.Output()


def search_jobs(b):
    with output:
        output.clear_output()
        company = company_input.value.strip() or None
        location = location_input.value.strip() or None
        job_title = job_title_input.value.strip() or None

        conn = None

        try:
            conn = get_db_connection()
            cur = conn.cursor()

            query = '''
            SELECT "JOB_ID", "COMPANY_NAME", "LOCATION", "TITLE" FROM public."POSTING"
            WHERE (%s IS NULL OR "COMPANY_NAME" ILIKE %s) 
            AND (%s IS NULL OR "LOCATION" ILIKE %s) 
            AND (%s IS NULL OR "TITLE" ILIKE %s)
            ORDER BY "JOB_ID" ASC
            LIMIT 10;
            '''

            # Prepare parameters
            params = (
                company, f"%{company}%" if company else None,
                location, f"%{location}%" if location else None,
                job_title, f"%{job_title}%" if job_title else None
            )

            cur.execute(query, params)
            rows = cur.fetchall()

            # Close cursor before processing results
            cur.close()
            conn.close()

            if rows:
                df = pd.DataFrame(rows, columns=["JOB_ID", "COMPANY_NAME", "LOCATION", "TITLE"])
                display(df)
            else:
                print("No job postings found with the selected parameters.")

        except psycopg2.Error as e:
            if conn:
                conn.rollback()  # Reset transaction state on error
            print(f"Database error: {e}")

        except Exception as e:
            print(f"Unexpected error: {e}")

        finally:
            if conn:
                conn.close()


search_button.on_click(search_jobs)

display(widgets.VBox([
    widgets.Label("Company Name:"), company_input,
    widgets.Label("Location:"), location_input,
    widgets.Label("Job Title:"), job_title_input,
    search_button, output
]))

## 3. Select all the job postings for the TechGiant (more than 500 employees and in the IT industry)

In [35]:
import ipywidgets as widgets
import pandas as pd
import psycopg2  # Ensure psycopg2 is installed

# Create input fields
industry_input = widgets.Text(placeholder="Insert the Industry domain")
company_input = widgets.Text(placeholder="Insert the company name")
search_button = widgets.Button(description="Look for job posting", button_style="primary")
output = widgets.Output()


# Function to search jobs in TechGiant companies
def search_jobs(b):
    with output:
        output.clear_output()
        company = company_input.value.strip() or None
        industry = industry_input.value.strip() or None

        conn = None

        try:
            conn = get_db_connection()
            cur = conn.cursor()

            query = '''
            SELECT P."COMPANY_NAME", P."VIEWS", P."PAY_PERIOD", P."APPLIES", P."EXPIRY", CI."INDUSTRY"
            FROM public."POSTING" as P 
            JOIN public."COMPANY_INDUSTRY" as CI ON P."COMPANY_ID" = CI."COMPANY_ID"
            JOIN public."EMPLOYEE_COUNTS" as EC ON P."COMPANY_ID" = EC."COMPANY_ID" 
            WHERE EC."EMPLOYEE_COUNT" > 2000 
            AND "VIEWS" IS NOT NULL
            AND (%s IS NULL OR CI."INDUSTRY" ILIKE %s)
            AND (%s IS NULL OR P."COMPANY_NAME" ILIKE %s) 
            ORDER BY "VIEWS" DESC
            LIMIT 20;
            '''

            # Prepare parameters (matching the placeholders)
            params = (
                industry, f"%{industry}%" if industry else None,
                company, f"%{company}%" if company else None
            )

            cur.execute(query, params)
            rows = cur.fetchall()

            # Close cursor before processing results
            cur.close()
            conn.close()

            if rows:
                df = pd.DataFrame(rows,
                                  columns=["COMPANY_NAME", "VIEWS", "PAY_PERIOD", "APPLIES", "EXPIRY", "INDUSTRY"])
                display(df)
            else:
                print("No job postings found for TechGiant companies.")

        except psycopg2.Error as e:
            if conn:
                conn.rollback()  # Reset transaction state on error
            print(f"Database error: {e}")

        except Exception as e:
            print(f"Unexpected error: {e}")

        finally:
            if conn:
                conn.close()


# Bind function to button click
search_button.on_click(search_jobs)

# Display UI
display(widgets.VBox([
    widgets.Label("Industry:"), industry_input,
    widgets.Label("Company Name:"), company_input,
    search_button, output
]))

## 4. Select all the companies and job position requested which salary is between two values given by input. Add info such as, all the prt-time with USD payment and Monthly pay period 

In [43]:
currency_input = widgets.Text(placeholder="Currency (e.g., USD)", description="Currency:")
job_type_input = widgets.Text(placeholder="Job Type (e.g., Part-time)", description="Job Type:")
pay_period_input = widgets.Text(placeholder="Pay Period (e.g., Monthly)", description="Pay Period:")

salary_slider = widgets.IntRangeSlider(
    value=[0, 10000],
    min=0,
    max=10000,
    step=100,
    description="Salary Range:",
    continuous_update=False
)

search_button = widgets.Button(description="Look for job postings", button_style="primary")
output = widgets.Output()

def search_jobs(b):
    with output:
        output.clear_output()
        currency = currency_input.value.strip() or None
        job_type = job_type_input.value.strip() or None
        pay_period = pay_period_input.value.strip() or None
        min_salary, max_salary = salary_slider.value  # Get range from slider

        conn = None

        try:
            conn = get_db_connection()
            cur = conn.cursor()

            query = '''
            SELECT P."COMPANY_NAME", P."TITLE", P."PAY_PERIOD", P."CURRENCY", P."WORK_TYPE", S."MIN_SALARY", S."MAX_SALARY"
            FROM public."POSTING" AS P JOIN public."SALARIES" AS S ON P."JOB_ID" = S."JOB_ID"
            WHERE
            (%s IS NULL OR P."CURRENCY" ILIKE %s)
            AND (%s IS NULL OR P."WORK_TYPE" ILIKE %s) 
            AND (%s IS NULL OR P."PAY_PERIOD" ILIKE %s)
            AND (S."MIN_SALARY" >= %s)
            AND (S."MAX_SALARY" <= %s)
            LIMIT 20;
            '''

            params = (
                currency, f"%{currency}%" if currency else None,
                job_type, f"%{job_type}%" if job_type else None,
                pay_period, f"%{pay_period}%" if pay_period else None,
                min_salary,
                max_salary
            )

            start_time = time.time()
            cur.execute(query, params)
            rows = cur.fetchall()
            end_time = time.time()
            
            execution_time = end_time - start_time

            cur.close()
            conn.close()

            print(f"Query executed in {execution_time:.4f} seconds")
            
            if rows:
                df = pd.DataFrame(rows, columns=["COMPANY_NAME", "JOB_TITLE", "PAY_PERIOD", "CURRENCY", "WORK_TYPE",
                                                 "MIN_SALARY", "MAX_SALARY"])
                display(df)
            else:
                print("No job postings found with the given salary range and conditions.")

        except psycopg2.Error as e:
            if conn:
                conn.rollback()
            print(f"Database error: {e}")

        except Exception as e:
            print(f"Unexpected error: {e}")

        finally:
            if conn:
                conn.close()

search_button.on_click(search_jobs)

display(widgets.VBox([
    widgets.Label("Enter Job Filters:"),
    currency_input,
    job_type_input,
    pay_period_input,
    salary_slider,
    search_button,
    output
]))

## 5. Considering today is today but one year early select all the active job postings that allows for remote working, that pay in USD, from companies with name given by input and that respect the characteristics of the logged user

## 6. Select all the ...

## 7. Select all the ...

## 8. Select all the ...

## 9. Select all the ...

## 10. Select all the ...

NOTE.
aggiungere experience_level alla tabella USER al posto di experience
Tramite una select prendere tutti i valori della colonna FORMATTED_EXPERIENCE_LEVEL dalla tabella posting e fare in modo che durante l inserimento da PYTHON puo essere scelto un unico valore tra questi della lista

aggiungere skill_id su tabella USER

prendere tutte le skill possibili (nome) dalla tabella SKILL e dare la possibilita all utente di inserire le proprie skill ognuna di queste sara una riga nella tabella di appoggio che verra creata USER_SKILL
Quando si cerca un job posting verranno visualizzate quelle che matchano almeno un numero da scegliere di skill richieste dal job (JOB_SKILLS) e quelle dell utente loggato o registrato

